In [10]:
from ImageLoader import *
from NoiseRemoval import *
from Segmentation import *
from FeatureExtractor import *
from Classifier import *
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
import bz2file as bz2
# plt.rc("figure", dpi=1000)

In [11]:
clf = None
Kmean = None

def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = pickle.load(data)
    return data


clf = decompress_pickle('models\\classifier2.pbz2')
if clf is not None:
    print("Classifier Loaded")
else:
    print("Failed to Load Classifier") 
    
with open('models/kmean.pkl', 'rb') as f:
    Kmean = pickle.load(f)
if Kmean is not None:
    print("Kmean Loaded")
else:
    print("Failed to Load Kmean") 

Classifier Loaded
Kmean Loaded


In [12]:
feature_extractor = FeatureExtractor()
classifier = Classifier()

In [13]:
if clf is None or Kmean is None:  
    dataset,labels = feature_extractor.loadDataset(".\\fonts-dataset\\")
    print("Loading DataSet Done")  
    X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=4000, random_state=42)
    print("Loading DataSet Done")     
    features = feature_extractor.extract(X_train,method = 'SIFT')
    print('Training Start')

    print(len(features))
    print(len(y_train))

    classifier.train(features,y_train)

    print('Training Done')

    with open('models/classifier.pkl', 'wb') as f:
        pickle.dump(classifier.pipeline, f)

    with open('models/kmean.pkl', 'wb') as f:
        pickle.dump(feature_extractor.cluster_model, f)

    print('Saving Model Done')
else:
    feature_extractor.cluster_model=Kmean
    classifier.pipeline=clf
    print('Model Loaded')

Model Loaded


In [8]:
predicted_labels = []
for i,image in enumerate(X_test):
    alteredImage=image
    alteredImage = NoiseRemoval.applyMedianFilter(image=image,kernel_size=3)
    alteredImage = Segmentation.segment(alteredImage)
    result = classifier.classify(alteredImage,feature_extractor,method='SIFT')
    predicted_labels.append(result)
        
accuracy = accuracy_score(y_test, predicted_labels)
print("Accuracy:", accuracy*100)

Accuracy: 97.37434358589647


In [14]:
images = ImageLoader.loadImages('self-generated-dataset\\3\\')

for image in images:
    alteredImage = NoiseRemoval.applyMedianFilter(image=image,kernel_size=3)
    alteredImage = Segmentation.segment(alteredImage)
    result = classifier.classify(alteredImage,feature_extractor,method='SIFT')
    if result == '4':
        print('Passed')
    else : 
        print('Ded')

Passed
Ded
Passed
Passed
Passed


In [15]:
images = ImageLoader.loadImages('Dataset\\1\\')

for image in images:
    alteredImage = NoiseRemoval.applyMedianFilter(image=image,kernel_size=3)
    alteredImage = Segmentation.segment(alteredImage)
    result = classifier.classify(alteredImage,feature_extractor,method='SIFT')
    if result == '1':
        print('Passed')
    else : 
        print('Ded')

Ded
Passed
Passed
Passed
Passed
Passed
Passed
Ded


In [3]:


def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        pickle.dump(data, f)

compressed_pickle('classifier2', clf)